In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import os
import copy
from xgboost import plot_importance, plot_tree, XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor



In [5]:
# os.chdir("")
# 1. PM2.5
dat = pd.read_csv("seoul_pm25.csv")
dat_miss = pd.read_csv("seoul_pm25_miss.csv")
# 2. PM10
dat = pd.read_csv("seoul_pm10.csv")
dat_miss = pd.read_csv("seoul_pm10_miss.csv")

pred_miss = copy.deepcopy(dat_miss)

test_m = ["2018-11","2019-02","2019-05","2019-08"]
train_m = [["2018-09","2018-10"],["2018-12","2019-01"],["2019-03","2019-04"],["2019-06","2019-07"]]

# tuning parameter # n_estimator control need
learn_rate = [0.05,0.1,0.3]
max_dep = [3,4,5,6]

for lr in learn_rate:
    for md in max_dep:
        mae, rmse = [[] for i in range(2)]
        n,m,r = [0 for i in range(3)]
        
        for i in range(4):
            train_dat = dat[dat["date"].str.slice(start=0,stop=7).isin(train_m[i])].iloc[:,2:].fillna(0)
            n_len = train_dat.shape[0]; tr_len = int(n_len*0.8) # 80% train, 20% validate
            
            for j in range(25):
                if j!=24 : 
                    train_x = train_dat.iloc[:tr_len,list(range(j))+list(range(1+j,25))]
                    test_x = train_dat.iloc[tr_len:,list(range(j))+list(range(1+j,25))]
                else : 
                    train_x = train_dat.iloc[:tr_len,range(24)]
                    test_x = train_dat.iloc[tr_len:,range(24)]
                    
                train_y = train_dat.iloc[:tr_len,j]
                test_y = train_dat.iloc[tr_len:,j]
                
                model = xgb.XGBRegressor(objective='reg:squarederror',learning_rate=lr, max_depth=md, seed=1234)
                model.fit(train_x, train_y)
                # predicting
                pred = model.predict(test_x)
                mae.append(abs(test_y - pred))
                rmse.append((test_y - pred)**2)
                
        for i in range(25):
            n += len(mae[i])
            m += np.sum(mae[i])
            r += np.sum(rmse[i])
        print("eta : {:0.2f}, max_dep : {:0.0f}, MAE : {:0.3f}, RMSE : {:0.3f}".format(lr,md,m/n,np.sqrt(r/n)))


eta : 0.05, max_dep : 3, MAE : 6.074, RMSE : 10.890
eta : 0.05, max_dep : 4, MAE : 6.157, RMSE : 10.989
eta : 0.05, max_dep : 5, MAE : 6.195, RMSE : 11.031
eta : 0.05, max_dep : 6, MAE : 6.294, RMSE : 11.108
eta : 0.10, max_dep : 3, MAE : 6.142, RMSE : 11.039
eta : 0.10, max_dep : 4, MAE : 6.135, RMSE : 10.997
eta : 0.10, max_dep : 5, MAE : 6.218, RMSE : 11.040
eta : 0.10, max_dep : 6, MAE : 6.265, RMSE : 11.087
eta : 0.30, max_dep : 3, MAE : 6.540, RMSE : 11.381


KeyboardInterrupt: 

In [4]:
# fitting               
for i in range(4):
    print(i)
    train_dat = dat[dat["date"].str.slice(start=0,stop=7).isin(train_m[i])].iloc[:,2:].fillna(0)
    test_dat = dat_miss[dat_miss["date"].str.slice(start=0,stop=7).isin([test_m[i]])].iloc[:,2:].fillna(0)
    for j in range(25):
        # fitting
        if j!=24 : 
            train_x = train_dat.iloc[:,list(range(j))+list(range(1+j,25))]
            test_x = test_dat.iloc[:,list(range(j))+list(range(1+j,25))]
        else : 
            train_x = train_dat.iloc[:,range(24)]
            test_x = test_dat.iloc[:,range(24)]
        train_y = train_dat.iloc[:,j]
        test_y = test_dat.iloc[:,j]
        model = xgb.XGBRegressor(objective='reg:squarederror',eta=0.05, max_depth=3, seed=1234)
        model.fit(train_x, train_y)
        # predicting
        pred_miss.iloc[pred_miss["date"].str.slice(start=0,stop=7).isin([test_m[i]]),2+j] = model.predict(test_x)

mae, rmse = [[] for i in range(2)]
for i in range(2,27):
    test_idx = (~np.isnan(np.array(dat.iloc[:,i], dtype=np.float))) ^ (~np.isnan(np.array(dat_miss.iloc[:,i],dtype=np.float)))
    mae.append(abs(pred_miss.iloc[test_idx,i] - dat.iloc[test_idx,i]))
    rmse.append((pred_miss.iloc[test_idx,i] - dat.iloc[test_idx,i])**2)

n,m,r=0,0,0
for i in range(25):
    n += len(mae[i])
    m += np.sum(mae[i])
    r += np.sum(rmse[i])

print("MAE : {:0.3f}, RMSE : {:0.3f}".format(m/n,np.sqrt(r/n)))


0
1
2
3
MAE : 10.413, RMSE : 18.661
